# Monads 2.0, aka Algebraic Effects: ExtensibleEffects.jl

an introduction to Monads, their downsides, and their next generation

by Stephan Sahm, stephan.sahm@gmx.de

## Outline
* Working with Monads
* Limitations of Monads
* Extensible Effects - How they work
* Extensible Effects - How to define your own
* Extensible Effects - more

<img src="images/jakobsweg-fisterra-fokus.png"
     alt="Stephan Sahm"
     style="float: right; margin-right: 3em"
     width="50%"/>

### Stephan Sahm

- freelancer
- end-to-end Data & AI consultant
- organizer of Julia User Group Munich

### key interests
- professional best practices
- probabilistic programming
- functional programming

# Let's start simple

The key idea behind Monads is to **encapsulate a computational context**
<br>
simplifying our daily programming

Loving context managers, let's encapsulate **construction (aka enter)** and **destruction (aka exit)**

In [1]:
struct MyContextManager
  implementation
end
(context::MyContextManager)(continuation) = context.implementation(continuation)

In [2]:
context = MyContextManager(continuation -> begin
    println("construction")
    x = 42
    result = continuation(x)
    println("destruction")
    result
end)

MyContextManager(var"#1#2"())

In [3]:
context(println)

construction
42
destruction


In [4]:
context() do inner_value
    println(inner_value)
    inner_value
end

construction
42
destruction


42

# How to work with such a computational context?

Situation: given a function `f` we want to apply it to our `42` encapsulated in our `context`.

😬 *on a first glance this looks impressively impossible*

😀 *but actually we can*

and by convention we store our logic in the higher-order function named `map`

In [5]:
Base.map(f, context::MyContextManager) = MyContextManager(continuation -> begin
  context() do inner_value
    continuation(f(inner_value))
  end
end)

In [6]:
context_doubled = map(x -> 2x, context)
context_doubled = map(context) do x
    2x
end
context_doubled(println)

construction
84
destruction


<div style="text-align: right"> 👏 it works 👏 </div>

# One step left: Combining multiple contexts

In [7]:
f_context(x) = MyContextManager(continuation -> begin
    println("before")
    y = 1111
    result = continuation(x+y)
    println("after")
    result
end)

context_in_context = map(f_context, context)
context_in_context(println)

construction
MyContextManager(var"#13#14"{Int64}(42))
destruction


we would like to **merge both contexts** together somehow... in literatur this merging is called *join* or *flatten*, hence...
<br>
such an enhanced `map` function is typically called `flatmap`

In [8]:
using TypeClasses
TypeClasses.flatmap(f_context, context::MyContextManager) = MyContextManager(continuation -> begin
    context() do inner_value
        inner_context = f_context(inner_value)
        inner_context(continuation)
    end
end)

In [9]:
context_flattened = flatmap(f_context, context)
context_flattened(println)

construction
before
1153
after
destruction


<div style="text-align: right"> 👏 it works 👏 </div>

# Making everything nice and convenient

That is it! Having `map` and `flatmap` defined, we get nice syntax for free, which just compiles to a combination of the two.

In [10]:
context_flattened = TypeClasses.@syntax_flatmap begin
    x = context
    f_context(x)
end
context_flattened(println)

construction
before
1153
after
destruction


In [11]:
context_flattened = TypeClasses.flatmap(((x,)->begin
    f_context(x)
end), context)
context_flattened(println)

construction
before
1153
after
destruction


within `@syntax_flatmap` every normal row is interpreted as context and `map`/`flatmap` are used.
<br>
You can also mix this with normal computation, just mark the normal computations by the macro `@pure`.

In [12]:
context_complex = @syntax_flatmap begin
    x = context
    @pure s = "x = $x"
    z = f_context(x)
    # we can define another final return value by ending with @pure
    @pure (x, s, z)
end
context_complex(println)

construction
before
(42, "x = 42", 1153)
after
destruction


In [13]:
context_complex = TypeClasses.flatmap(((x,)->begin
    s = "x = $(x)"
    TypeClasses.map(((z,)->begin
        (x, s, z)
    end), f_context(x))
end), context)
context_complex(println)

construction
before
(42, "x = 42", 1153)
after
destruction


# Limitation of Monads: they don't compose well

Here we have two monads (aka computational context): `Vector` and `ContextManager`

In [14]:
# Vector are supported by TypeClasses
# with for-loop interpretation
@syntax_flatmap begin
    a = [3, 7]
    b = [100, 200]
    @pure a + b
end

4-element Vector{Int64}:
 103
 203
 107
 207

In [15]:
# MyContextManager is fully implemented in TypeClasses
create_context(x) = @ContextManager continuation -> begin
    println("before $x")
    result = continuation(x)
    println("after $x")
    result
end
context = @syntax_flatmap begin
    a = create_context(3)
    b = create_context(10)
    @pure a + b
end
context(println)

before 3
before 10
13
after 10
after 3


Let's compose them!

In [16]:
@syntax_flatmap begin
    a = [100, 200]
    # contextmanager is converted to Vector by simply running it
    b = create_context(a)
    @pure a + b
end

before 100
after 100
before 200
after 200


2-element Vector{Int64}:
 200
 400

In [17]:
context_composition = @syntax_flatmap begin
    a = create_context(7)
    b = [100, 200]
    @pure a + b
end
context_composition(println)

before 7


LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mVector{Int64}[39m[0m to an object of type 
[0m  [91mContextManager[39m
[0mClosest candidates are:
[0m  convert(::Type{var"#s9"} where var"#s9"<:ContextManager, [91m::Identity[39m) at /home/ssahm/.julia/dev/DataTypesBasic/src/convert.jl:34
[0m  convert(::Type{var"#s28"} where var"#s28"<:ContextManager, [91m::Writer[39m) at /home/ssahm/.julia/dev/TypeClasses/src/convert.jl:18
[0m  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:205
[0m  ...

# ExtensibleEffects.jl

There is a lot of theory developed around this problem of composing computational contexts.
- **Monad-Transformers:** difficult to port to Julia, and don't compose well
- **Algebraic / Extensible Effects:** compose impressively well, but also have some limitations

ExtensibleEffects.jl implements Extensible Effects following the paper [Freer Monads, More Extensible Effects](http://okmij.org/ftp/Haskell/extensible/more.pdf) which already has an [Haskell implementation](https://hackage.haskell.org/package/freer-effects) as well as a [Scala implementation](https://github.com/atnos-org/eff).

In [18]:
using ExtensibleEffects
context_composition = @runcontextmanager @runhandlers (Vector,) @syntax_eff_noautorun begin
    a = create_context(7)
    b = [100, 200]
    @pure a + b
end
context_composition(println)

before 7
[107, 207]
after 7


In [19]:
context_composition2 = @runcontextmanager @runhandlers (Vector,) @syntax_eff_noautorun begin
    a = [100, 200]
    b = create_context(a)
    @pure a + b
end
context_composition2(println)

before 100
before 200
[200, 400]
after 200
after 100


<div style="text-align: right"> 👏 it works 👏 </div>

# How does it work?

using `@macroexpand` we can see what is happening

In [20]:
eff = @syntax_eff_noautorun begin
    a = create_context(7)
    b = [100, 200]
    @pure a + b
end

Eff(value=ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(7)), length(cont)=1)

In [21]:
TypeClasses.flatmap(((a,)->begin
    TypeClasses.map(((b,)->begin
        a + b
    end), ExtensibleEffects.effect([100, 200]))
end), ExtensibleEffects.effect(create_context(7)))

Eff(value=ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(7)), length(cont)=1)

-------------

- [same as for `@syntax_flatmap`] <br> every `=` is parsed as `flatmap` or `map`
- [**NEW**] every effect is wrapped with `ExtensibleEffects.effect`

In [22]:
# effect makes sure we have an Eff type
ExtensibleEffects.effect([100, 200])

Eff(value=[100, 200], length(cont)=0)

# Everything is wrapped into the type `Eff` for later ("lazy") execution

# Nothing is executed first

# Eff

The `Eff` type is very simple: It just stores the **value** and the **continuation** which leads to the next `Eff`.

In [23]:
# slight simplification of the real type
struct Eff′{Effectful, Continuation}
    value::Effectful
    cont::Continuation  # just a function, which returns another Eff
end
# for performance improvements the `Continuation` is internally represented 
# as a list of functions which get composed (instead of a single composed function)

# Running Eff - Handlers

The effects can be evaluated in kind of every order (not 100% true, but almost)

In [24]:
@show eff.value; println(); @show eff.cont;

eff.value = ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(7))

eff.cont = ExtensibleEffects.Continuation{Tuple{var"#61#63"}}((var"#61#63"(),))


In [25]:
eff′ = @runhandlers (Vector,) eff

@show eff′.value; println(); @show eff′.cont;

eff′.value = ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(7))

eff′.cont = ExtensibleEffects.Continuation{Tuple{ExtensibleEffects.var"#6#8"{UnionAll, ExtensibleEffects.Eff{ContextManager{var"#35#36"{Int64}}, Tuple{var"#61#63"}}}}}((ExtensibleEffects.var"#6#8"{UnionAll, ExtensibleEffects.Eff{ContextManager{var"#35#36"{Int64}}, Tuple{var"#61#63"}}}(Vector{T} where T, Eff(value=ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(7)), length(cont)=1)),))


----------------
- the value has not changed at all
- the continuation changed, namely, **now every `Vector` will get executed in this continuation**

# Running Eff - Handlers 2

Let's see what happens if the first effect is `Vector`

In [26]:
eff2 = @syntax_eff_noautorun begin
    a = [100, 200]
    b = create_context(a)
    @pure a + b
end
@show eff2.value; println(); @show length(eff2.cont.functions);

eff2.value = [100, 200]

length(eff2.cont.functions) = 1


In [27]:
eff2′ = @runhandlers (Vector,) eff2

@show eff2′.value
println()
@show length(eff2′.cont.functions);

eff2′.value = ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(100))

length(eff2′.cont.functions) = 5


----------------
- the value has changed
    - [**NEW**] the original `Vector` was handled. The next unhandled effect appears, here the `ContextManager`
- the continuation changed
    - [same as before] every `Vector` will get executed in this continuation
    - [**NEW**] one for each vector-element there is one separate continuation, which all are combined into one overall continuation

# What happened when we handled the original Vector?

1. the Vector is `[100, 200]`

In [28]:
a = [100, 200]

2-element Vector{Int64}:
 100
 200

2. for each element we run the continuation, each continuation returns an `Eff`

In [29]:
continuation(a) = @syntax_eff_noautorun begin
    b = create_context(a)
    @pure a + b
end
a′ = map(continuation, a);

3. so we have one `Eff` encapsulating the result for 100, and another `Eff` which encapsulates the result for 200

In [30]:
(a′100, a′200) = a′

2-element Vector{ExtensibleEffects.Eff{ContextManager{var"#35#36"{Int64}}, Tuple{ComposedFunction{typeof(noeffect), var"#73#74"{Int64}}}}}:
 Eff(value=ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(100)), length(cont)=1)
 Eff(value=ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(200)), length(cont)=1)

4. treating `Eff` as computational context (aka Monad) in its own right, we can work **within** `Eff` (just like we have worked within `ContextManager`) and combine both `Eff` to a combined effect

In [31]:
@syntax_flatmap begin
    x = a′100
    y = a′200
    @pure [x, y]
end

Eff(value=ContextManager{var"#35#36"{Int64}}(var"#35#36"{Int64}(100)), length(cont)=2)

(The real implementation is slightly different, because Vector can be of arbitrary length, but you get the point.)

# The Interface - How to define your own Effects?

core function | description
    ------------- | ------------
    `ExtensibleEffects.eff_applies(handler::Type{<:Vector}, vec::Vector) = true` | specify on which values the handler applies (the handler Vector applies to Vector of course)
    `ExtensibleEffects.eff_pure(handler::Type{<:Vector}, value) = [value]` | wrap a plain value into the Monad of the handler, here Vector.
    `ExtensibleEffects.eff_flatmap(continuation, vec::Vector)` | apply a continuation to the current effect (here again Vector as an example). The key difference to plain `TypeClasses.flatmap` is that `continuation` does not return a plain `Vector`, but a `Eff{Vector}`. Applying this `continuation` with a plain `map` would lead `Vector{Eff{Vector}}`. However, `eff_flatmap` needs to return an `Eff{Vector}` instead.

Thanks to `TypeClasses.jl` we can provide a generic interface which works out of the box for most types

```julia
function ExtensibleEffects.eff_pure(T, a)
  TypeClasses.pure(T, a)
end

function ExtensibleEffects.eff_flatmap(continuation, a)
  a_of_eff_of_a = map(continuation, a)
  eff_of_a_of_a = TypeClasses.flip_types(a_of_eff_of_a)
  eff_of_a = map(TypeClasses.flatten, eff_of_a_of_a)
  eff_of_a
end
```

`TypeClasses.flatten` just wraps `TypeClasses.flatmap`

For implementing `TypeClasses.flip_types`, if your type supports `Base.iterate` and (only needed in case of multiple elements) also `TypeClasses.combine`, you can simply fallback to `TypeClasses.default_flip_types_...`) 

Consequently, here the actual implementation of `Vector`


core function | description
------------- | ------------
`ExtensibleEffects.eff_applies(handler::Type{<:Vector}, vec::Vector) = true` | always needs to be provided explicitly
`Base.map(f, vec::Vector) = [f(x) for x in vec]` | apply a function within the Monad.
`Base.iterate(vec::Vector, state...) = ...` | standard iteration protocol 
`TypeClasses.pure(handler::Type{<:Vector}, value) = [value]` | wrap a plain value into the Monad of the handler, here Vector.
`TypeClasses.flatmap(f, vec::Vector) = [x for vec2 in vec for x in vec2]` | apply a function returning a Monad and flatten everything immediately.
`TypeClasses.combine(vec1::Vector, vec2::Vector) = [vec1; vec2]` | !!Only needed if your Monad contains multiple elements!! Combining two values of the same Monad
`TypeClasses.flip_types(vec::Vector) = TypeClasses.default_flip_types_having_pure_combine_apEltype(vec)` | make a vector_of_something into something_of_vector 

It is the recommended set of functions to be implemented, if possible for your type. 


#### If this is not possible, you can
* fallback up to providing a direct implementation of `eff_flatmap` for your type
* implement a custom handler for your type containing additionally needed information

# More Extensible Effects - autorun

This packages comes with the super awesome functionality that standard effects can be run automatically without explicitly calling the handler.

This is implemented by using the type of the effect itself as the handler (plus some smart evaluation logic what to do in case no such handler exists.)

* `@syntax_eff` enables `autorun`
* `@syntax_eff_noautorun` disables `autorun`


In [32]:
@syntax_eff begin
  x = [1,4]
  [x, 3x]
end

4-element Vector{Int64}:
  1
  3
  4
 12

# More Extensible Effects - contextmanager

Turns out, `ContextManager` is actually one of the most difficult computational contexts to be represented within Extensible Effects.
<br>
This is because, within `Eff` the continuations always return another `Eff`, within which other effects still need to be handled.

Consequently, the contextmanager handler `ContextManagerHandler` **needs to run last**.

----------------

There is further a special handler `ContextManagerCombinedHandler` which improves the execution order for memory performance.

In [33]:
println_return(x) = (println(x); x)
eff = @syntax_eff begin
  a = [100,200]
  b = create_context(a)
  c = [5,6]
  d = create_context(a + c)
  @pure a, b, c, d
end
@runhandlers (ContextManagerHandler(println_return),) eff

before 100
before 105
before 106
before 200
before 205
before 206
[(100, 100, 5, 105), (100, 100, 6, 106), (200, 200, 5, 205), (200, 200, 6, 206)]
after 206
after 205
after 200
after 106
after 105
after 100


4-element Vector{NTuple{4, Int64}}:
 (100, 100, 5, 105)
 (100, 100, 6, 106)
 (200, 200, 5, 205)
 (200, 200, 6, 206)

In [34]:
eff = @syntax_eff noautorun(Vector) begin
  a = [100,200]
  b = create_context(a)
  c = [5,6]
  d = create_context(a + c)
  @pure a, b, c, d
end
handlers = (ContextManagerCombinedHandler(Vector, println_return),)
@runhandlers handlers eff  

before 100
before 105
(100, 100, 5, 105)
after 105
before 106
(100, 100, 6, 106)
after 106
after 100
before 200
before 205
(200, 200, 5, 205)
after 205
before 206
(200, 200, 6, 206)
after 206
after 200


4-element Vector{NTuple{4, Int64}}:
 (100, 100, 5, 105)
 (100, 100, 6, 106)
 (200, 200, 5, 205)
 (200, 200, 6, 206)

# More Extensible Effects - More Effects

ExtensibleEffects supports many more types:

types defined in `DataTypesBasic.jl`
* Option
* Try
* Either
* Identity
* Const
* ContextManager

types defined in `TypeClasses.jl`
* Callable
* Iterable
* State
* Writer

types defined in `Base`
* Vector
* Task
* Future

# Summary

* Monads = easy encapsulation of an computational context
* `map` + `flatmap` and you get nice syntax support (powered by `TypeClasses.jl`)
* Monads do not compose well with other Monads
* Extensible Effects do compose well
* They do so by defining a meta-Monad called `Eff` which just stores everything for lazy execution later on.
* handlers define how certain Monads/Effects are actually run
* thanks to `autorun` you don't need to bother about handlers for most cases
* `ContextManagerHandler` or `ContextManagerCombinedHandler` always needs to be the last handler to be run
* many Monads are already implemented and work out of the box - take a look at the documentation and tests

# Thank you for your attention

Find more help at the repositories and documentations respectivly
* https://github.com/JuliaFunctional/ExtensibleEffects.jl
* https://github.com/JuliaFunctional/TypeClasses.jl
* https://github.com/JuliaFunctional/DataTypesBasic.jl

The code and tests are all easy to understand, don't hesitate to take a look in case of questions!

<img src="images/jakobsweg-fisterra-fokus.png"
     alt="Stephan Sahm"
     style="float:left"
     width="20%"/>

<div style="margin-left: 7em !important">
You are always welcome to reach out

- github: schlichtanders
- mail: stephan.sahm@gmx.de
- linkedin: https://de.linkedin.com/in/stephan-sahm-918656b7
</div>
